In [1]:
# Librerías
import pandas as pd 
import numpy
import matplotlib.pyplot as plt 
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

In [2]:
spark = SparkSession.builder \
     .master("local") \
     .appName("OPI_B1") \
     .config("spark.some.config.option", "some-value") \
     .getOrCreate()

In [3]:
df = spark.read.csv('all_data.csv', header='true')
df.columns

['producto',
 'presentacion',
 'marca',
 'categoria',
 'catalogo',
 'precio',
 'fechaRegistro',
 'cadenaComercial',
 'giro',
 'nombreComercial',
 'direccion',
 'estado',
 'municipio',
 'latitud',
 'longitud']

### 1. Procesamiento de los datos
a. ¿Cuántos registros hay?

La base de datos está compuesta por 62,530,715 registos 

In [18]:
df.createOrReplaceTempView('Tabla')
spark.sql(
'''
SELECT COUNT(*) 
FROM Tabla
'''
).show()

+--------+
|count(1)|
+--------+
|62530715|
+--------+



b. ¿Cuántas categorías?

Existen 41 categorías distintas

In [16]:
spark.sql(
'''
SELECT COUNT(DISTINCT categoria) 
FROM Tabla
'''
).show()

+-------------------------+
|count(DISTINCT categoria)|
+-------------------------+
|                       41|
+-------------------------+



c. ¿Cuántas cadenas comerciales están siendo monitoreadas?

A falta de saber alguna regla de negocio para saber cuáles cadenas comerciales están siendo monitoreadas, consideraré que todas las cadenas comerciales que aparecen en la base de datos están siendo monitoreadas: 705

In [17]:
spark.sql(
'''
SELECT COUNT(DISTINCT cadenaComercial) 
FROM Tabla
'''
).show()

+-------------------------------+
|count(DISTINCT cadenaComercial)|
+-------------------------------+
|                            705|
+-------------------------------+



d. ¿Cómo podrías determinar la calidad de los datos? 

Nos podemos ayudar mediante el tipo de dato que se supone está contenido en cada columna, me explico:

Si el deber ser es que la columna "precio" acepte únicamente valores numéricos, podríamos hacer una función de agregado, como por ejemplo un sum() con group by(), si el query finaliza correctamente, significa que no hay ningún caracter diferente a un número en esa columna, por lo que podríamos considerarla limpia (posiblemente haya ceros, habría que averiguar si eso es válido).

De manera similar con las columna fechaRegistro, podríamos aplicarle un to_date() para tratar de dejar la columna en formato date, si todos los registros están capturados con el mismo formato de la máscara que especifiquemos, podremos considerar que la columna está limpia en cuanto a estructura (la validez de cada fecha registrada ya es un tema aparte).

Eso podría ser desde un punto de vista técnico

Otras maneras implican tener un conocimiento más profundo sobre el negocio para analizar la información y ver que no se esté violando ninguna regla de negocio. 

¿Detectaste algún tipo de inconsistencia o error en la fuente?

Sí, conforme se avanza en los ejercicios es posible notar que los valores en la tabla están chuecos, me di cuenta que el municipio aparece en el campo de estado en algunos casos, además de que hay algunos valores nulos.

e. ¿Cuáles son los productos más monitoreados en cada entidad?

Debajo del siguiente chunk de código, se muestra una tabla con los productos más monitoreados en cada entidad (asumiendo que por el simple hecho de estar en la tabla ya está siendo monitoreado).

In [32]:
spark.sql(
'''
SELECT contador, producto, estado FROM (
        SELECT COUNT(*) as contador
                , producto
                , estado 
        FROM Tabla 
        GROUP BY producto, estado
        ) A
        WHERE A.contador = (SELECT MAX(contador) FROM (SELECT COUNT(*) as contador
                                                                , producto
                                                                , estado 
                                                        FROM Tabla 
                                                        GROUP BY producto, estado) B
                            WHERE A.estado=B.estado)
'''
).show(truncate=False)
# Por lo visto, el límite de spark es de 20 filas para mostrar, tenemos que quitar este límite
# para poder ver los registros de los 32 estados

+--------+-----------------------+--------------------+
|contador|producto               |estado              |
+--------+-----------------------+--------------------+
|43904   |REFRESCO               |TLAXCALA            |
|20      |producto               |estado              |
|27868   |LECHE ULTRAPASTEURIZADA|QUERÉTARO           |
|50307   |DETERGENTE P/ROPA      |NUEVO LEÓN          |
|35991   |LECHE ULTRAPASTEURIZADA|YUCATÁN             |
|18137   |REFRESCO               |HIDALGO             |
|8003    |REFRESCO               |NAYARIT             |
|20884   |DETERGENTE P/ROPA      |ZACATECAS           |
|27770   |REFRESCO               |BAJA CALIFORNIA SUR |
|40144   |DETERGENTE P/ROPA      |MICHOACÁN DE OCAMPO |
|17193   |DETERGENTE P/ROPA      |COLIMA              |
|15982   |DETERGENTE P/ROPA      |CHIHUAHUA           |
|31597   |REFRESCO               |PUEBLA              |
|275     |REFRESCO               | COL. EDUARDO GUERRA|
|81718   |REFRESCO               |JALISCO       

f. ¿Cuál es la cadena comercial con mayor variedad de productos monitoreados?

De nuevo es importante definir a qué nos referimos cuando decimos variedad, porque una cadena comercial puede tener diversos tipos de productos pero bajo la misma categoría (como ABASTECEDORA LUMEN, que bajo la categoría de "MATERIAL ESCOLAR" tiene muchos productos). Pese a que los productos son diferentes, todos pertenecen a la misma categoría. 

Por lo tanto vamos a plantear los 2 escenarios:

1) Diversidad cuando una cadena tiene diferentes categorías de productos (no es de sorprender que las cadenas de supermercados son las que aparecieron con la mayor diversidad, presentando un empate en primer lugar 18 cadenas):

In [46]:
spark.sql(
'''
SELECT COUNT(*) contador, A.cadenaComercial FROM (
                                        SELECT categoria, cadenaComercial FROM Tabla
                                        GROUP BY categoria, cadenaComercial
                                        ) A
        GROUP BY cadenaComercial
        ORDER BY contador desc
'''
).show()

+--------+--------------------+
|contador|     cadenaComercial|
+--------+--------------------+
|      41|            CHEDRAUI|
|      41|              H.E.B.|
|      41|            WAL-MART|
|      41|        SORIANA PLUS|
|      41|BODEGA COMERCIAL ...|
|      41|            I.M.S.S.|
|      41|  COMERCIAL MEXICANA|
|      41|  LEY (AUTOSERVICIO)|
|      41|        I.S.S.S.T.E.|
|      41|     MERCADO SORIANA|
|      41|      MEGA COMERCIAL|
|      41|             SORIANA|
|      41|       SORIANA SUPER|
|      41|            CASA LEY|
|      41|MEGA COMERCIAL ME...|
|      41|HIPERMERCADO SORIANA|
|      41|      BODEGA AURRERA|
|      41|   SUPERMERCADOS LEY|
|      40|            SUPERAMA|
|      40|              S MART|
+--------+--------------------+
only showing top 20 rows



2) Por otra parte, si tomamos como diversidad de productos al producto como tal sin imoprtar la categoría, de nueva cuenta aparecen las grandes cadenas de super-mercados (en este caso SORIANA quedó en primer lugar)

In [47]:
spark.sql(
'''
SELECT COUNT(*) contador, A.cadenaComercial FROM (
                                        SELECT producto, cadenaComercial FROM Tabla
                                        GROUP BY producto, cadenaComercial
                                        ) A
        GROUP BY cadenaComercial
        ORDER BY contador desc
'''
).show()

+--------+--------------------+
|contador|     cadenaComercial|
+--------+--------------------+
|    1059|             SORIANA|
|    1051|            WAL-MART|
|    1049|MEGA COMERCIAL ME...|
|    1036|  COMERCIAL MEXICANA|
|    1026|            CHEDRAUI|
|    1024|     MERCADO SORIANA|
|    1012|      BODEGA AURRERA|
|    1006|HIPERMERCADO SORIANA|
|    1001|              H.E.B.|
|     999|        SORIANA PLUS|
|     996|       SORIANA SUPER|
|     979|BODEGA COMERCIAL ...|
|     937|        I.S.S.S.T.E.|
|     936|            SUPERAMA|
|     851|              S MART|
|     849|SUPERMERCADOS SAN...|
|     848|              SUMESA|
|     844|         CITY MARKET|
|     819|FARMACIA GUADALAJARA|
|     808|            CASA LEY|
+--------+--------------------+
only showing top 20 rows



### 2. Análisis exploratorio
a. Genera una canasta de productos básicos que te permita comparar los precios
geográfica y temporalmente. Justifica tu elección y procedimiento

Mi elección se basó en tomar algunos de los elementos de la canasta básica en México que extraje de la siguiente liga: https://www.animalgourmet.com/2019/01/30/canasta-basica-mexico/

La lista completa es
1. Maíz
2. Azúcar
3. Frijol
4. Arroz
5. Harina de maíz
6. Aceite vegetal
7. Chiles jalapeños, rajas y chipotles
8. Leche en polvo
9. Atún
10. Sardina
11. Sal de mesa
12. Café soluble
13. Chocolate en polvo
14. Galletas marías, de animalitos o saladas
15. Harina de trigo
16. Pasta para sopa
17. Avena
18. Lentejas
19. Detergente en polvo
20. Jabón de lavandería
21. Pasta de dientes
22. Jabón de tocador
23. Papel higiénico

Y los que tomaremos para la prueba son:

1. Maíz
4. Arroz
8. Leche en polvo
12. Café soluble
16. Pasta para sopa
17. Avena
20. Jabón de lavandería
21. Pasta de dientes
22. Jabón de tocador
23. Papel higiénico

Mi elección de estos productos, es porque considero que dichos productos cubren los aspectos de: higiene y comida. 

Mi procedimiento será ejecutar queries a la base de datos buscando por el nombre del producto e ir ajustando el nombre del producto conforme vaya obteniendo resultados e ir echando un vistazo a cómo está la base

In [81]:
spark.sql(
'''
SELECT producto, estado, AVG(precio) FROM Tabla
WHERE producto = 'TORTILLA DE MAIZ'
AND estado NOT IN ('COL. EDUARDO GUERRA',' COL. EDUARDO GUERRA')
GROUP BY producto, estado
ORDER BY estado
'''
).show(40)

+----------------+--------------------+---------------------------+
|        producto|              estado|avg(CAST(precio AS DOUBLE))|
+----------------+--------------------+---------------------------+
|TORTILLA DE MAIZ|       ESQ. SUR 125"|                       11.0|
|TORTILLA DE MAIZ|      AGUASCALIENTES|         10.677146171693737|
|TORTILLA DE MAIZ|     BAJA CALIFORNIA|         13.178771820448878|
|TORTILLA DE MAIZ| BAJA CALIFORNIA SUR|         12.865167498218108|
|TORTILLA DE MAIZ|            CAMPECHE|         12.204763918252286|
|TORTILLA DE MAIZ|             CHIAPAS|         11.000996282527877|
|TORTILLA DE MAIZ|           CHIHUAHUA|         11.921376068376066|
|TORTILLA DE MAIZ|COAHUILA DE ZARAGOZA|         11.091259961231955|
|TORTILLA DE MAIZ|              COLIMA|          12.02032338014872|
|TORTILLA DE MAIZ|    DISTRITO FEDERAL|         10.500537028660721|
|TORTILLA DE MAIZ|             DURANGO|         10.764948006932404|
|TORTILLA DE MAIZ|          GUANAJUATO|         

In [82]:
spark.sql(
'''
SELECT producto, estado, AVG(precio) FROM Tabla
WHERE producto = 'ARROZ'
AND estado NOT IN ('COL. EDUARDO GUERRA',' COL. EDUARDO GUERRA')
AND estado is not NULL
GROUP BY producto, estado
ORDER BY estado
'''
).show(40)

+--------+--------------------+---------------------------+
|producto|              estado|avg(CAST(precio AS DOUBLE))|
+--------+--------------------+---------------------------+
|   ARROZ|      AGUASCALIENTES|          15.44374545454545|
|   ARROZ|     BAJA CALIFORNIA|          16.31233333333333|
|   ARROZ| BAJA CALIFORNIA SUR|         15.515476595744678|
|   ARROZ|            CAMPECHE|         15.003935802469131|
|   ARROZ|             CHIAPAS|          14.46720151014629|
|   ARROZ|           CHIHUAHUA|         15.341243105642763|
|   ARROZ|COAHUILA DE ZARAGOZA|          15.62060876161577|
|   ARROZ|              COLIMA|         15.838562241616913|
|   ARROZ|    DISTRITO FEDERAL|         16.234147984968274|
|   ARROZ|             DURANGO|         15.743647604327663|
|   ARROZ|          GUANAJUATO|         16.199112157390275|
|   ARROZ|            GUERRERO|         16.007272131147538|
|   ARROZ|             HIDALGO|         16.405257270693514|
|   ARROZ|             JALISCO|         

In [83]:
spark.sql(
'''
SELECT producto, estado, avg(precio) FROM Tabla
WHERE producto = 'JABON DE TOCADOR'
AND estado is not NULL
AND estado NOT IN ('COL. EDUARDO GUERRA',' COL. EDUARDO GUERRA')
GROUP BY producto, estado
ORDER BY estado
'''
).show(40)

+----------------+--------------------+---------------------------+
|        producto|              estado|avg(CAST(precio AS DOUBLE))|
+----------------+--------------------+---------------------------+
|JABON DE TOCADOR|      AGUASCALIENTES|         11.629121756487029|
|JABON DE TOCADOR|     BAJA CALIFORNIA|         12.803907237846097|
|JABON DE TOCADOR| BAJA CALIFORNIA SUR|         12.679502304867576|
|JABON DE TOCADOR|            CAMPECHE|         12.178285827919924|
|JABON DE TOCADOR|             CHIAPAS|          11.84136334651193|
|JABON DE TOCADOR|           CHIHUAHUA|         12.533713620488948|
|JABON DE TOCADOR|COAHUILA DE ZARAGOZA|         12.050133912956593|
|JABON DE TOCADOR|              COLIMA|         11.724857873364416|
|JABON DE TOCADOR|    DISTRITO FEDERAL|         11.562145678232921|
|JABON DE TOCADOR|             DURANGO|         12.191551812805344|
|JABON DE TOCADOR|          GUANAJUATO|          11.56959048827834|
|JABON DE TOCADOR|            GUERRERO|         

In [84]:
spark.sql(
'''
SELECT producto, estado, AVG(precio) FROM Tabla
WHERE producto = 'CAFE SOLUBLE'
AND estado is not NULL
AND estado NOT IN ('COL. EDUARDO GUERRA',' COL. EDUARDO GUERRA')
GROUP BY producto, estado
ORDER BY estado
'''
).show(40)

+------------+--------------------+---------------------------+
|    producto|              estado|avg(CAST(precio AS DOUBLE))|
+------------+--------------------+---------------------------+
|CAFE SOLUBLE|      AGUASCALIENTES|          47.23466049990159|
|CAFE SOLUBLE|     BAJA CALIFORNIA|          49.87659131363473|
|CAFE SOLUBLE| BAJA CALIFORNIA SUR|         49.453608582574766|
|CAFE SOLUBLE|            CAMPECHE|         47.306037186379925|
|CAFE SOLUBLE|             CHIAPAS|         47.793080151908875|
|CAFE SOLUBLE|           CHIHUAHUA|          50.41494650410547|
|CAFE SOLUBLE|COAHUILA DE ZARAGOZA|          50.11465650120159|
|CAFE SOLUBLE|              COLIMA|           49.7774469849544|
|CAFE SOLUBLE|    DISTRITO FEDERAL|         49.348395190980305|
|CAFE SOLUBLE|             DURANGO|         49.434653218495036|
|CAFE SOLUBLE|          GUANAJUATO|          48.33522773582646|
|CAFE SOLUBLE|            GUERRERO|          46.71030919220058|
|CAFE SOLUBLE|             HIDALGO|     

In [80]:
spark.sql(
'''
SELECT producto, estado, avg(precio) FROM Tabla
WHERE producto = 'AVENA'
AND estado is not NULL
AND estado NOT IN ('COL. EDUARDO GUERRA',' COL. EDUARDO GUERRA')
GROUP BY producto, estado
ORDER BY estado
'''
).show(40);

+--------+--------------------+---------------------------+
|producto|              estado|avg(CAST(precio AS DOUBLE))|
+--------+--------------------+---------------------------+
|   AVENA|      AGUASCALIENTES|         20.796043956043953|
|   AVENA|     BAJA CALIFORNIA|         23.463725828068164|
|   AVENA| BAJA CALIFORNIA SUR|         23.447436905932477|
|   AVENA|            CAMPECHE|         22.347740585774055|
|   AVENA|             CHIAPAS|          22.68960089938168|
|   AVENA|           CHIHUAHUA|         25.182150878203277|
|   AVENA|COAHUILA DE ZARAGOZA|         23.145824526420736|
|   AVENA|              COLIMA|         21.297950377562014|
|   AVENA|    DISTRITO FEDERAL|           20.6520749733191|
|   AVENA|             DURANGO|         20.467755466309665|
|   AVENA|          GUANAJUATO|         20.110086313193587|
|   AVENA|            GUERRERO|         22.897755406413125|
|   AVENA|             HIDALGO|         18.975141395908555|
|   AVENA|             JALISCO|         

De los 10 artículos que tenía contemplados para la canasta básica, decidí dejarlo en 5, debido a la tardanza de ejecución de los queries (no me parecen queries muy complejos, pero a mi máquina tal vez le está costando trabajar con esa base).

Los 5 productos que decidí lejar son los siguientes:

1. AVENA
2. CAFE SOLUBLE
3. ABON DE TOCADOR
4. ARROZ
5. TORTILLA DE MAIZ

Y mediante el siguiente query se obtendrá el promedio de precio de cada producto por estad, y posteriormente se sumarán dichos promedios para obtener el precio de lo que costaría la canasta básica antes mencionada.

In [85]:
spark.sql(
'''
SELECT A.estado, sum(A.precio) FROM (
                SELECT producto, estado, avg(precio) precio FROM Tabla
                WHERE producto IN ('AVENA','CAFE SOLUBLE','ABON DE TOCADOR','ARROZ','TORTILLA DE MAIZ')
                AND estado is not NULL
                AND estado NOT IN ('COL. EDUARDO GUERRA',' COL. EDUARDO GUERRA')
                GROUP BY producto, estado
                ORDER BY estado
                ) A
GROUP BY A.estado
'''
).show(40);

+--------------------+------------------+
|              estado|       sum(precio)|
+--------------------+------------------+
|       ESQ. SUR 125"|              11.0|
|      AGUASCALIENTES| 94.15159608218472|
|     BAJA CALIFORNIA|102.83142229548508|
| BAJA CALIFORNIA SUR|101.28168958247002|
|            CAMPECHE| 96.86247749287539|
|             CHIAPAS| 95.95087884396472|
|           CHIHUAHUA|102.85971655632757|
|COAHUILA DE ZARAGOZA| 99.97234975047006|
|              COLIMA| 98.93428298428206|
|    DISTRITO FEDERAL|  96.7351551779284|
|             DURANGO| 96.41100429606477|
|          GUANAJUATO| 95.06434013801463|
|            GUERRERO| 97.20851417130183|
|             HIDALGO| 94.11566774218903|
|             JALISCO| 95.40190318967684|
| MICHOACÁN DE OCAMPO| 96.93540731398114|
|             MORELOS| 98.11885284704292|
|              MÉXICO|  95.3244420743568|
|             NAYARIT| 98.42873247911105|
|          NUEVO LEÓN| 99.59352727851481|
|              OAXACA| 94.53651369

b. ¿Cuál es la ciudad más cara del país? ¿Cuál es la más barata?

Conforme a la consulta anterior, la ciudad más cara del país en cuanto a productos es Sonora, mientras que la más barata es aguascalientes

c. ¿Hay algún patrón estacional entre años?

En general a lo largo del año van subiendo los precios de todos los productos, por lo que a inicio de año es más común encontrar "precios bajos" en comparación a los precios que se manejarán en diciembre.

Esto se debe a los ajustes inflacionarios que se realizan a lo largo del año, los cuales prácticamente son una constante, en ese sentido, no sé si cuente como patrón estacional puesto que siempre tiende hacia arriba.

En el siguiente chunk de código se muestra el query correspondiente a este punto

In [101]:
spark.sql(
'''
SELECT AVG(A.precio) precio, a.producto, A.estado, A.anio, A.mes FROM (
        SELECT producto
        , precio
        , estado
        --, fechaRegistro
        , SUBSTR(fechaRegistro,1,4) ANIO
        , SUBSTR(fechaRegistro,6,2) MES
        FROM Tabla
        WHERE estado='DISTRITO FEDERAL'
        AND producto IN ('AVENA','CAFE SOLUBLE','JABON DE TOCADOR','ARROZ','TORTILLA DE MAIZ')
) A
GROUP BY A.producto, A.estado, A.anio, A.mes
ORDER BY A.anio, A.mes, A.estado, A.producto
LIMIT 60
'''
).show(60)

+------------------+----------------+----------------+----+---+
|            precio|        producto|          estado|anio|mes|
+------------------+----------------+----------------+----+---+
|14.790818713450292|           ARROZ|DISTRITO FEDERAL|2011| 01|
|17.211827411167512|           AVENA|DISTRITO FEDERAL|2011| 01|
| 42.60177215189874|    CAFE SOLUBLE|DISTRITO FEDERAL|2011| 01|
| 9.726635325721961|JABON DE TOCADOR|DISTRITO FEDERAL|2011| 01|
| 8.459247648902823|TORTILLA DE MAIZ|DISTRITO FEDERAL|2011| 01|
|14.316445182724253|           ARROZ|DISTRITO FEDERAL|2011| 02|
|16.894213333333337|           AVENA|DISTRITO FEDERAL|2011| 02|
| 44.56351774530271|    CAFE SOLUBLE|DISTRITO FEDERAL|2011| 02|
| 9.746635220125789|JABON DE TOCADOR|DISTRITO FEDERAL|2011| 02|
| 8.551330798479087|TORTILLA DE MAIZ|DISTRITO FEDERAL|2011| 02|
|14.462762148337594|           ARROZ|DISTRITO FEDERAL|2011| 03|
|17.429297872340424|           AVENA|DISTRITO FEDERAL|2011| 03|
|  44.9780172413793|    CAFE SOLUBLE|DIS

d. ¿Cuál es el estado más caro y en qué mes?

Retomando el punto b en el que mencionamos que Sonora era el estado más caro, y el punto c en el que se menciona que a lo largo del año el alza de los precios es una constante, el mes más caro es diciembre.

e. ¿Cuáles son los principales riesgos de hacer análisis de series de tiempo con
estos datos?

De entrada la base está sucia, pese a que los campos sucios de momento sólo se me han presentado en campos distintos a la fecha, no dudo que dicho campo también pueda estar mal en algunos casos.


### 3. Visualización

a. Genera un mapa que nos permita identificar la oferta de categorías en la zona metropolitana de León Guanajuato y el nivel de precios en cada una de ellas. Se darán puntos extra si el mapa es interactivo